# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [419]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [420]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split



In [421]:
def build_classifiers():
    return(
        LogisticRegression(max_iter=10000).fit(data_set, labels),
        KNeighborsClassifier().fit(data_set, labels),
        SVC(kernel="linear").fit(data_set, labels),
        DecisionTreeClassifier().fit(data_set, labels),
        GaussianNB().fit(data_set, labels),
        QuadraticDiscriminantAnalysis().fit(data_set, labels)
    )

In [422]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    stacked_classifier = DecisionTreeClassifier()
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [423]:
classifiers = build_classifiers()
predicted = build_stacked_classifier(classifiers[:3])
accuracy = accuracy_score(test_labels, predicted)
print(accuracy)

0.8


In [424]:
bestAccuracy = 0
best3Classifiers = None
for classifierA in classifiers:
    for classifierB in classifiers:
        for classifierC in classifiers:
            if classifierA == classifierB or classifierA == classifierC or classifierB == classifierC:
                continue
            predicted = build_stacked_classifier([classifierA, classifierB, classifierC])
            accuracy = accuracy_score(test_labels, predicted)
            if accuracy > bestAccuracy:
                bestAccuracy = accuracy
                best3Classifiers = [classifierA, classifierB, classifierC]
            # print("Accuracy: ", [classifierA, classifierB, classifierC], " acc : ", accuracy)

print("Best accuracy: ", bestAccuracy)
print("Best classifiers: ", best3Classifiers)

Best accuracy:  0.95
Best classifiers:  [DecisionTreeClassifier(), KNeighborsClassifier(), QuadraticDiscriminantAnalysis()]


## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [425]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0



def calculate_accuracy_vector(predicted, labels):
    result = []
    for i in range(len(predicted)):
        if predicted[i] == labels[i]:
            result.append(0)
        else:
            result.append(1)
    return np.array(result)



Fill the two functions below:

In [426]:
def set_new_weights(model):
    I =  calculate_accuracy_vector(model.predict(test_set), test_labels)
    return (1 + I) / (1 + I).sum()


Train the classifier with the code below:

In [448]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

validate_x, validate_label = generate_data(1, dimension, labels)

Set the validation data set:

In [444]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [445]:

def get_prediction(x):
    predictions = np.array([classifier.predict(x) for classifier in classifiers])
    predicted = []
    for i in range(len(x)):
        classified = predictions[:, i]
        counts = np.bincount(classified)
        predicted.append(np.argmax(counts))
    return predicted

Test it:

In [449]:
prediction = get_prediction(validate_x)[0]
print(prediction)

1
